### Stock Recommender based on signals from WallStreetBets

#### Get top stocks from WSB for given month

In [ ]:
import praw

# Insert your PRAW credentials here 
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    password="",
    user_agent="",
    username="",check_for_async=False
)
print(reddit.user.me())

diamond_hands_007


In [ ]:
def get_stock_list():
    ticker_dict = {}
    filelist = ['/content/drive/MyDrive/unstructured_project/ticker_list/list1.csv', 
                '/content/drive/MyDrive/unstructured_project/ticker_list/list2.csv',
                '/content/drive/MyDrive/unstructured_project/ticker_list/list3.csv']
    for file in filelist:
        tl = pd.read_csv(file, skiprows=0, skip_blank_lines=True)
        tl = tl[tl.columns[0]].tolist()
        for ticker in tl:
            ticker_dict[ticker] = 1
    return ticker_dict

def get_tickers(sub, stockList):
    monthlyTickers = {}
    regexPattern = r'\b([A-Z]+)\b'
    tickerDict = stockList
    blacklist = ["A", "I", "DD", "WSB", "YOLO", "RH", "EV", "PE", "ETH", "BTC", "E"] 
    for submission in reddit.subreddit(sub).top("month"):
        strings = [submission.title]
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            strings.append(comment.body)
        for s in strings:
            for phrase in re.findall(regexPattern, s):
                if phrase not in blacklist:
                    if phrase in tickerDict:
                        if phrase not in monthlyTickers:
                            monthlyTickers[phrase] = 1
                        else:
                            monthlyTickers[phrase] += 1
    return monthlyTickers

stockList = get_stock_list()
topTickers = {}
for sub in ['wallstreetbets']:
    monthlyTickers = get_tickers(sub, stockList)
    for ticker in monthlyTickers.keys():
        if ticker in topTickers:
            topTickers[ticker] += monthlyTickers[ticker]
        else:
            topTickers[ticker] = monthlyTickers[ticker]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.


In [ ]:
ticker_freq = pd.DataFrame(sorted(topTickers.items(), key=lambda x: x[1], reverse=True))
ticker_freq.columns = ['ticker', 'tf']
ticker_freq.to_csv('data/top_tickers_30days.csv')
ticker_freq = pd.read_csv('data/top_tickers_30days.csv', index_col=0)

In [ ]:
# Filter out single character tickers
ticker_freq = ticker_freq[ticker_freq['ticker'].apply(lambda x:len(x) > 1)]

In [ ]:
# Manually removing some unwanted words that will not be tickers
set_tickers = set(ticker_freq[:50]['ticker'].tolist()).difference(set(['FOR', ' U', 'ALL', 'CAN', 'EDIT', 'GDP', 'CEO', 'CFO', 'NYC', 'ARE', 'FSD', 'NOW', 'IMO', 'USA', 'USB', 'QQQ', 'ONE',
                                                         'MSM', 'JOE', 'IRS', 'ANY', 'ATH', 'BOOM', 'UK']))
ticker_freq = ticker_freq[ticker_freq['ticker'].isin(list(set_tickers))]

### Perform Sentiment Analysis on scraped posts and comments

In [ ]:
import praw
import pandas as pd
import os
import datetime
import requests
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
wsb = reddit.subreddit('wallstreetbets')

Common WSB WSB slang/terminology and associated sentiments added below. These lexicons are assigned based on the topic modelling we did. Essentially its a bear market so the negative sentiment is prevalent.

In [ ]:
wsb_words = {
    'tendies': 2.0,
    'bull': 2.0,
    'bear': -10.0,
    'short': -10.0,
    'sell': -10.0,
    'puts': -10.0,
    'calls': 2.0,
    'moon': 2.0,
    'btfd': 2.0,
    'going tits up': -10.0,
}

In [ ]:
def get_sentiment(df_col, words=wsb_words):

    # initializes VADER
    vader = SentimentIntensityAnalyzer()

    # updates VADER's lexicon to include WSB terminology
    vader.lexicon.update(wsb_words)

    # analyzes sentiment of dataframe column input
    scores = df_col.apply(vader.polarity_scores).tolist()
    return pd.DataFrame(scores)

def get_date(submission):
    
    # gets submission datetime
    try:
        time = submission.created
    except: 
        time = submission
    return datetime.date.fromtimestamp(time)

def scrape_for(symbol, limit=10, before='', after='30d', subreddit='wallstreetbets'): #considered 30 days as this will take data from 1st september to date, if we want to look at a wider timeframe, change here
    result = []
    url_for_comment = 'https://api.pushshift.io/reddit/search/comment/?q={0}&subreddit={1}&after={2}&size={3}&before={4}&sort_type=score'.format(symbol, subreddit, after, limit, before)
    comments = requests.get(url_for_comment).json()['data']

    url_for_submission = 'https://api.pushshift.io/reddit/search/submission/?q={0}&subreddit={1}&after={2}&size={3}&before={4}&sort_type=score'.format(symbol, subreddit, after, limit, before)
    submissions = requests.get(url_for_submission).json()['data']

    for comment in comments:
        result.append((comment['body'], get_date(comment['created_utc'])))

    for submission in submissions:
        date = get_date(submission['created_utc'])
        try:
            result.append((submission['title'] + submission['selftext'], date))
        except:
            result.append((submission['title'], date))
        sub = reddit.submission(id=submission['id'])
        sub.comments.replace_more(limit=0)
        for top_level_comment in sub.comments:
            result.append((top_level_comment.body, date))

    # creates a dataframe from API results
    df = pd.DataFrame(result, columns=['Text', 'Date'])

    # updates dataframe to include sentiments
    scored_df = df.join(get_sentiment(df['Text']), rsuffix='_right')

    return scored_df

In [ ]:
# Get compund sentiment for top n stocks from WSB as per VADER sentiment analysis

top_n = 5 
ticker_freq['compound_sentiment'] = 0
for ticker in ticker_freq[:top_n]['ticker'].tolist():
  sentiment_total = scrape_for(ticker)
  ticker_freq['compound_sentiment'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()

In [ ]:
# sentiment for top n stocks (over last 30 days)
ticker_freq[:top_n]

,ticker,tf,compound_sentiment
0,GME,382,0.029451
2,AI,127,-0.022523
3,BBBY,101,-0.026281
5,AMC,90,-0.049026
6,CS,85,-0.111998


In [ ]:
# dollar has almost neutral
sentiment_total = scrape_for('USD')
sentiment_total['compound'].mean()

-0.04046829268292681

In [ ]:
# pound has a negative sentiment
sentiment_total = scrape_for('sterling')
sentiment_total['compound'].mean()

-0.136907

### Stock Simulator - analysis in restrospect

In [ ]:
# Get top_n recommended stocks as per the WallStreetBets sentiment

top_n = 10
ticker_freq['compound_sentiment'] = 0
ticker_freq['compound_sentiment_1month_back'] = 0
ticker_freq['compound_sentiment_2months_back'] = 0
for ticker in ticker_freq[:top_n]['ticker'].tolist():
  sentiment_total = scrape_for(ticker, after='90d', before='60d') #3 months back with 1 month data
  ticker_freq['compound_sentiment_2months_back'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()
  sentiment_total = scrape_for(ticker, after='60d', before='30d') #2 months back with 1 month data
  ticker_freq['compound_sentiment_1month_back'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()
  sentiment_total = scrape_for(ticker) #1 months back with 1 month data
  ticker_freq['compound_sentiment'][ticker_freq['ticker'] == ticker] = sentiment_total['compound'].mean()

In [ ]:
ticker_freq[:top_n]

,ticker,tf,compound_sentiment,compound_sentiment_1month_back,compound_sentiment_2months_back
0,GME,382,0.029451,-0.034960,-0.035763
2,AI,127,-0.022523,0.094232,0.144990
3,BBBY,101,-0.026281,0.007149,-0.000798
5,AMC,90,-0.049026,0.004049,0.056591
6,CS,85,-0.111998,0.149132,0.096831
9,TSLA,63,-0.011124,-0.082407,-0.052972
14,IT,44,-0.094143,-0.041592,-0.036135
18,NVDA,37,-0.062092,-0.078903,0.043307
19,GM,36,0.045823,0.184702,-0.107337
23,AMD,34,-0.007595,-0.032588,-0.009890


In [ ]:
# Get stock prices from 1-month back from yahoo finance to compare with our recommendations 

# !pip install yahoo_fin
from yahoo_fin.stock_info import get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table

for ticker in ticker_freq[:top_n]['ticker'].tolist():
  print(get_data(ticker, start_date = '07/01/2022' , end_date = '10/01/2022' , interval = '1mo')[['ticker', 'close']])

           ticker      close
2022-07-01    GME  34.009998
2022-08-01    GME  28.639999
2022-09-01    GME  25.129999
           ticker  close
2022-07-01     AI  18.41
2022-08-01     AI  18.00
2022-09-01     AI  12.50
           ticker  close
2022-07-01   BBBY   5.03
2022-08-01   BBBY   9.53
2022-09-01   BBBY   6.09
           ticker  close
2022-07-01    AMC  14.56
2022-08-01    AMC   9.12
2022-09-01    AMC   6.97
           ticker  close
2022-07-01     CS   5.80
2022-08-01     CS   5.16
2022-09-01     CS   3.92
           ticker       close
2022-07-01   TSLA  297.149994
2022-08-01   TSLA  275.609985
2022-09-01   TSLA  265.250000
           ticker       close
2022-07-01     IT  265.480011
2022-08-01     IT  285.320007
2022-09-01     IT  276.690002
           ticker       close
2022-07-01   NVDA  181.630005
2022-08-01   NVDA  150.940002
2022-09-01   NVDA  121.389999
           ticker      close
2022-07-01     GM  36.259998
2022-08-01     GM  38.209999
2022-09-01     GM  32.090000
        

Out of the top-10 recommended stocks, we would have lost money only on C3.AI (AI) and Credit Suisse (CS) and made money on the remaining 8 stocks